In [ ]:

# import the necessary packages
from imutils.video import VideoStream
from imutils.video import FPS
import argparse
import time
import cv2
import os
from google.colab.patches import cv2_imshow
import imutils

In [ ]:
!pip3 uninstall opencv-python
!pip3 uninstall opencv-python-inference-engine
!pip3 install opencv-python-inference-engine

Found existing installation: opencv-python 4.6.0.66
Uninstalling opencv-python-4.6.0.66:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python-4.6.0.66.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Core-39545cc7.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Gui-48e93776.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Test-c38a5234.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5Widgets-e69d94fb.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libQt5XcbQpa-c112ba75.so.5.15.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libX11-xcb-69166bdf.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libXau-00ec42fe.so.6.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python.libs/libavcodec-5896f664.so.58.134.100
    /usr/local/lib/python3.7/d

In [ ]:
!pip install opencv-contrib-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
v_path="/content/drive/MyDrive/Colab Notebooks/openvino_vehicle_detection/result/video.mp4"
vs = cv2.VideoCapture(v_path)
if vs :
    print("Accessing Video Stream")
height = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))
print("Video resolution : ", height, width)

fourcc = cv2.VideoWriter_fourcc(*'XVID')

assert vs.isOpened(), 'Cannot capture source'
#Add extension mask_rcnn to video name
video_detection_path =  v_path.split(".")[0] + "_result_pedestrian-and-vehicle-detector." + v_path.split(".")[1]


#Video save
videoSaved = cv2.VideoWriter(video_detection_path, apiPreference=cv2.CAP_FFMPEG,
                             fourcc = fourcc,fps =20, frameSize=(width, height),	
                             isColor= True)


net = cv2.dnn.readNet('/content/drive/MyDrive/Colab Notebooks/openvino_vehicle_detection/Models/pedestrian-and-vehicle-detector-adas-0001.xml', '/content/drive/MyDrive/Colab Notebooks/openvino_vehicle_detection/Models/pedestrian-and-vehicle-detector-adas-0001.bin')

while vs.isOpened() :
  grabbed , frame  = vs.read()
  if frame is None:
    break
  #frame = imutils.resize(frame, width=300)
  image_ = frame.copy()


  # if the frame not grabbing means that we reach the end
  if not grabbed :
    print("No frame read from the steam")
    break
  H, W, _ = frame.shape
  #prediction
  # Prepare input blob and perform an inference
  blob = cv2.dnn.blobFromImage(frame, size=(672, 384), ddepth=cv2.CV_8U)
  net.setInput(blob)
  out = net.forward()

  predictions = []

  # The net outputs a blob with the shape: [1, 1, N, 7], where N is the number of detected bounding boxes.
  # For each detection, the description has the format: [image_id, label, conf, x_min, y_min, x_max, y_max]

  # Draw detected faces on the frame
  for detection in out.reshape(-1, 7):
      image_id, label, conf, x_min, y_min, x_max, y_max = detection

      if conf > 0.8:
          pred_boxpts = ((x_min, y_min), (x_max, y_max))

          # create prediciton tuple and append the prediction to the
          # predictions list
          prediction = (label, conf, pred_boxpts)
          predictions.append(prediction)
  if predictions :
        # loop over our predictions
        for (i, pred) in enumerate(predictions):
            # extract prediction data for readability
            (label, pred_conf, pred_boxpts) = pred
            ((x_min, y_min), (x_max, y_max)) = pred_boxpts 
            
            # filter out weak detections by ensuring the `confidence`
            # is greater than the minimum confidence
           # if pred_conf > 0.8:
                # print prediction to terminal
               # print("[INFO] Prediction #{}: confidence={}, "
               #       "boxpoints={}".format(i, pred_conf,
                 #                           pred_boxpts))

                # check if we should show the prediction data
                # on the frame
                #if args["display"] > 0:
                    # build a label
            if label == 1:
                label = "vehicle: {:.2f}%".format(pred_conf * 100)
            elif label == 2:
                label = "pedestrian: {:.2f}%".format(pred_conf * 100)


            x_min = int(x_min * W)
            y_min = int(y_min * H)
            x_max = int(x_max * W)
            y_max = int(y_max * H)

            # extract information from the prediction boxpoints
            y = y_min - 15 if y_min - 15 > 15 else y_min + 15

            # display the rectangle and label text
            cv2.rectangle(image_, (x_min, y_min), (x_max, y_max),
                          (255, 0, 0), 2)
            cv2.putText(image_, label, (x_min, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

        videoSaved.write(image_)
       # cv2_imshow(image_)

videoSaved.release()


 